In [ ]:
# default_exp policy_gradient

# Policy Gradient

> Policy gradient modules

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *

In [ ]:
# export

class BasePolicy():
    def __init__(self, gamma=1.):
        self.gamma = gamma

    def discount_rewards(self, rewards, mask, traj_rewards):
        rewards = scatter_rewards(rewards, mask)

        if traj_rewards is not None:
            rewards += traj_rewards

        discounted = discount_rewards(rewards, self.gamma)

        return discounted

In [ ]:
# export

class PolicyGradient(BasePolicy):
    def __init__(self, discount=True, gamma=0.97, ratio=False, scale_rewards=True):
        super().__init__(gamma)
        self.discount = discount
        self.ratio = ratio
        self.scale_rewards = scale_rewards
        self.mean_reward = None
        
    def __call__(self, lps, mask, rewards, base_lps=None, traj_rewards=None):
        if self.ratio:
            lps = (lps - base_lps.detach()).exp()
            
        if not self.discount:
            pg_loss = -((lps*mask).sum(-1)*rewards)/mask.sum(-1)
            
        else:
            rewards = self.discount_rewards(rewards, mask, traj_rewards)
            rewards = whiten(rewards, mask=mask)
            pg_loss = -(lps*rewards*mask).sum(-1)/mask.sum(-1)
            
        pg_dict = {'loss':pg_loss.detach().cpu(), 
                   'rewards':rewards.detach().cpu()}
            
        return pg_loss, pg_dict
    
    def from_batch_state(self, batch_state):
        lps = batch_state.model_gathered_logprobs
        base_lps = batch_state.base_gathered_logprobs
        mask = batch_state.mask
        
        rewards = batch_state.rewards
        
        if self.scale_rewards:
            if self.mean_reward is None:
                self.mean_reward = rewards.mean()
            else:
                self.mean_reward = (1-self.gamma)*rewards.mean() + self.gamma*self.mean_reward
                
            rewards = rewards - self.mean_reward
        
        traj_rewards = batch_state.trajectory_rewards
        
        loss, pg_dict = self(lps, mask, rewards, base_lps, traj_rewards)
        return loss, pg_dict


In [ ]:
# export

class TRPO(BasePolicy):
    def __init__(self, gamma, kl_target, beta=1., eta=50, lam=0.95, 
                 v_coef=0.5, scale_rewards=True):
        self.gamma = gamma
        self.beta = beta
        self.eta = eta
        self.lam = lam
        self.kl_target = kl_target
        self.v_coef = v_coef
        self.scale_rewards = scale_rewards
        self.mean_reward = None
        
    def __call__(self, lps_g, base_lps_g, lps, base_lps, mask, 
                 rewards, values, traj_rewards=None):
    
        discounted_rewards = self.discount_rewards(rewards, mask, traj_rewards)
        advantages = self.compute_advantages(discounted_rewards, values)
        advantages = whiten(advantages, mask=mask)
        
        v_loss = self.value_loss(values, discounted_rewards)
        
        ratios = (lps_g - base_lps_g.detach()).exp()
        
        loss1 = -(ratios*advantages*mask).sum(-1)/mask.sum(-1)
        
        kl = torch.distributions.kl.kl_divergence(
                    Categorical(logits=base_lps),
                    Categorical(logits=lps))
        
        kl = (kl*mask).sum(-1)/mask.sum(-1)
        loss2 = self.beta*kl
        
        loss3 = self.eta * torch.maximum(to_device(torch.tensor(0.)), 
                                         kl - 2.0*self.kl_target)

        pg_loss = loss1 + loss2 + loss3 + v_loss.mean(-1)
        
        pg_dict = { 'loss' : pg_loss.detach().cpu(),
                    'pg_discounted' : discounted_rewards.detach().cpu(),
                    'pg_advantage' : advantages.detach().cpu(),
                    'ratios' : ratios.detach().cpu(),
                    'kl' : kl.detach().cpu(),
                    'loss1' : loss1.detach().cpu(),
                    'loss2' : loss2.detach().cpu(),
                    'loss3' : loss3.detach().cpu(),
                    'v_loss' : v_loss.detach().cpu()}
        
        return pg_loss, pg_dict
    
    def from_batch_state(self, batch_state):
        lps_g = batch_state.model_gathered_logprobs
        base_lps_g = batch_state.base_gathered_logprobs
        
        lps = batch_state.model_logprobs
        base_lps = batch_state.base_logprobs
        
        mask = batch_state.mask
        rewards = batch_state.rewards
        
        if self.scale_rewards:
            if self.mean_reward is None:
                self.mean_reward = rewards.mean()
            else:
                self.mean_reward = (1-self.gamma)*rewards.mean() + self.gamma*self.mean_reward
                
            rewards = rewards - self.mean_reward
        
        traj_rewards = batch_state.trajectory_rewards
        
        values = batch_state.state_values
        
        loss, pg_dict = self(lps_g, base_lps_g, lps, base_lps, mask, 
                             rewards, values, traj_rewards)
        return loss, pg_dict

    def compute_advantages(self, rewards, values):

        if values is None:
            advantages = rewards
        else:
            advantages = compute_advantages(rewards, values.detach(), self.gamma, self.lam)

        return advantages

    def value_loss(self, values, rewards):
        if values is None:
            v_loss = to_device(torch.tensor(0.))
        else:
            v_loss = self.v_coef*F.mse_loss(values, rewards, reduction='none')

        return v_loss

In [ ]:
# export   

class PPO(BasePolicy):
    def __init__(self, gamma, kl_coef, lam=0.95, v_coef=0.5, cliprange=0.2, 
                 v_cliprange=0.2, ent_coef=0.01, kl_target=None, 
                 kl_horizon=None, scale_rewards=True):
        self.gamma = gamma
        self.lam = lam
        self.ent_coef = ent_coef
        self.kl_coef = kl_coef
        self.kl_target = kl_target
        self.kl_horizon = kl_horizon
        self.v_coef = v_coef
        self.cliprange = cliprange
        self.v_cliprange = v_cliprange
        self.scale_rewards = scale_rewards
        self.mean_reward = None
        
    def __call__(self, lps, base_lps, mask, 
                 rewards, values, ref_values, traj_rewards=None):
        
        discounted_rewards = self.discount_rewards(rewards, mask, traj_rewards)
        kl_reward = self.compute_kl_reward(lps, base_lps)
        
        discounted_rewards = discounted_rewards + kl_reward
        advantages = self.compute_advantages(discounted_rewards, values)
        advantages = whiten(advantages, mask=mask)
        
        v_loss = self.value_loss(values, ref_values, discounted_rewards)
        
        ratios = (lps - base_lps).exp()
        ratios_clipped = torch.clamp(ratios, 1.0-self.cliprange, 1.0+self.cliprange)
        
        loss1 = -(ratios*advantages)
        loss2 = -(ratios_clipped*advantages)
        
        loss = torch.maximum(loss1, loss2)
        loss = (loss*mask).sum(-1)/mask.sum(-1)
        
        entropy = Categorical(lps).entropy()
                
        pg_loss = loss + v_loss.mean(-1) - self.ent_coef*entropy
        
        self.update_kl(lps, base_lps, mask)
         
        pg_dict = { 'loss' : pg_loss.detach().cpu(),
                    'pg_discounted' : discounted_rewards,
                    'pg_advantage' : advantages,
                    'ratios' : ratios.detach().cpu(),
                    'loss' : loss.detach().cpu(),
                    'v_loss' : v_loss.detach().cpu(),
                    'entropy' : entropy.detach().cpu()}
        
        return pg_loss, pg_dict
    
    def from_batch_state(self, batch_state):
        lps = batch_state.model_gathered_logprobs
        base_lps = batch_state.base_gathered_logprobs
        
        
        mask = batch_state.mask
        rewards = batch_state.rewards
        
        if self.scale_rewards:
            if self.mean_reward is None:
                self.mean_reward = rewards.mean()
            else:
                self.mean_reward = (1-self.gamma)*rewards.mean() + self.gamma*self.mean_reward
                
            rewards = rewards - self.mean_reward
            
        traj_rewards = batch_state.trajectory_rewards
        
        values = batch_state.state_values
        ref_values = batch_state.ref_state_values
        
        loss, pg_dict = self(lps, base_lps, mask, rewards, 
                             values, ref_values, traj_rewards)
        
        return loss, pg_dict
            
    def compute_kl_reward(self, lps, base_lps):
        kl = lps - base_lps
        kl_reward = -self.kl_coef * kl.detach()
        return kl_reward
    
    def value_loss(self, values, old_values, rewards):
        if values is None:
            v_loss = to_device(torch.tensor(0.))
        else:
            
            v_loss = F.mse_loss(values, rewards, reduction='none')
            
            if old_values is not None:
                min_v = old_values - self.v_cliprange
                max_v = old_values + self.v_cliprange
                
                values_clipped = torch.max(torch.min(values, max_v), min_v)
                v_loss2 = F.mse_loss(values_clipped, rewards, reduction='none')
                
                v_loss = torch.max(v_loss, v_loss2)
            
            v_loss = self.v_coef*v_loss
            
        return v_loss
    
    def compute_advantages(self, rewards, values):
        
        if values is None:
            advantages = rewards
        else:
            advantages = compute_advantages(rewards, values.detach(), self.gamma, self.lam)
            
        return advantages
    
    def update_kl(self, lps, base_lps, mask):
        
        if (self.kl_target is not None) and (self.kl_horizon is not None):
            kl = (lps - base_lps).detach()
            kl = (kl*mask).sum(-1)/mask.sum(-1)
            kl = kl.cpu().mean()
            
            error = torch.clip(kl/self.kl_target - 1, -0.2, 0.2)
            factor = 1 + error * lps.shape[0]/self.kl_horizon
            self.kl_coef *= factor
    

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()